# The Battle of Neighborhoods - New York City

## Methodology and Coding

Below libraries are imported for fetching and reading datasets.

In [157]:

import pandas as pd # library for data analsysis
import numpy as np
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # To convert address into coordinates
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')



Libraries imported.


For New York City's Boroughs, Neighborhoods and their coordinate values, we use the below JSON file.

In [158]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


Let us read the JSON file content.

In [159]:
with open('newyork_data.json') as json_data:
    newyork_data=json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Load the JSON file content into pandas dataframe.

In [160]:

# Extract only the features section from json file
neighborhoods_data=newyork_data['features']


# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)



for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let us process only Bronx Borough for simplicity.

In [161]:
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
Bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [162]:
print('Bronx Borough is having {} neighborhoods'.format(Bronx_data.shape[0]))

Bronx Borough is having 52 neighborhoods


Now that we have location data of neighborhoods, using this we are going to get the italian restaurants in these neighborhoods since our client Michael needs a job in Italian restaurants. For this we are going to use Foursquare API.

In [163]:
# The code was removed by Watson Studio for sharing.

In Foursquare, the category ID for italian restaurants is 4bf58dd8d48988d110941735. We are going to use that in our foursquare url to get the list of italian restaurants. below function returns us the list of italian restaurants from Foursquare.

In [164]:
def get_italian_restaurants(neighborhood_latitude,neighborhood_longitude):
    LIMIT = 100 # limit of number of venues returned by Foursquare API

    radius = 2000 # define radius
    # create URL
    url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d110941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT) 
    IRList=[]
    results = requests.get(url).json()
    IRcount= len(results['response']['venues'])
    
    for i in range(IRcount):
        Irestaurant=results['response']['venues'][i]['name']
        IRList.append(Irestaurant)
    
    return IRList

Below is the sample function returns us the ratings of a restaurant based on it's venue id.

In [165]:
def get_rating(venueid,neighborhood_latitude,neighborhood_longitude):
    venue_url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
    venueid,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude
    )
    
    venue_results = requests.get(venue_url).json()
    
    try:
        restaurant_rating = venue_results['response']['venue']['rating']
    except:
        restaurant_rating = 0
    
    restaurant_name = venue_results['response']['venue']['name']
    
    return restaurant_name, restaurant_rating


Since the venue details end_point of foursquare api is very much limited (premium call), I could not get the ratings of all the restaurants. So, instead of using the above function, I am going to randomly generate the restaurant ratings from 0 to 10. 
I am going to get the total number of italian restaurants in a neighborhood. Then I am going to calculate the average rating of italian restaurants in a neighborhood. Also I am grouping the restaurants based on the ratings and getting counts such as High(rating >= 7), Medium(7> rating >=4) and low(4> rating >0).

In [166]:
# Iterate through the rows of Bronx dataframe for each neighborhood
for row in range(Bronx_data.shape[0]):
    
    neighborhood_latitude = Bronx_data.loc[row, 'Latitude'] # neighborhood latitude value
    neighborhood_longitude = Bronx_data.loc[row, 'Longitude'] # neighborhood longitude value
    neighborhood_name = Bronx_data.loc[row, 'Neighborhood'] # neighborhood name

    IRList=[]
    
    # Get the list of restaurants in a particular neighborhood
    IRList=get_italian_restaurants(neighborhood_latitude,neighborhood_longitude)
    
    # The length of the restaurants list is obtained as count
    Bronx_data.loc[row, 'Count'] = int(len(IRList))
    
    # Random numpy method is used for generating ratings for each restaurants
    Rating=np.random.randint(low=0, high=10, size=len(IRList))
    
    # Average rating is calculated
    RatingAverage=np.mean(Rating)
    
    Bronx_data.loc[row, 'Average Rating'] = round(RatingAverage,2)
    
    # The restaurants are grouped by ratings and the counts are derived in each group
    HighRatings=0
    LowRatings=0
    MidRatings=0
    
    for i in Rating:
        if i >= 7:
            HighRatings+=1
        elif i >= 4:
            MidRatings+=1
        else:
            LowRatings+=1
    
    Bronx_data.loc[row, 'High Rated Total'] = HighRatings
    Bronx_data.loc[row, 'Mid Rated Total'] = MidRatings
    Bronx_data.loc[row, 'Low Rated Total'] = LowRatings
    

Bronx_data.head(10)
    

,Borough,Neighborhood,Latitude,Longitude,Count,Average Rating,High Rated Total,Mid Rated Total,Low Rated Total
0,Bronx,Wakefield,40.894705,-73.847201,9.0,4.56,3.0,2.0,4.0
1,Bronx,Co-op City,40.874294,-73.829939,6.0,4.83,2.0,2.0,2.0
2,Bronx,Eastchester,40.887556,-73.827806,1.0,4.00,0.0,1.0,0.0
3,Bronx,Fieldston,40.895437,-73.905643,17.0,5.00,8.0,2.0,7.0
4,Bronx,Riverdale,40.890834,-73.912585,15.0,4.33,5.0,3.0,7.0
5,Bronx,Kingsbridge,40.881687,-73.902818,19.0,4.63,7.0,4.0,8.0
6,Bronx,Woodlawn,40.898273,-73.867315,14.0,4.93,7.0,2.0,5.0
7,Bronx,Norwood,40.877224,-73.879391,11.0,5.82,5.0,4.0,2.0
8,Bronx,Williamsbridge,40.881039,-73.857446,4.0,5.00,1.0,2.0,1.0
9,Bronx,Baychester,40.866858,-73.835798,14.0,4.93,5.0,4.0,5.0


In [167]:
# Let us drop the Borough column, since all rows are of Bronx Borough
Bronx_data=Bronx_data.drop('Borough',1)
Bronx_data.head()

,Neighborhood,Latitude,Longitude,Count,Average Rating,High Rated Total,Mid Rated Total,Low Rated Total
0,Wakefield,40.894705,-73.847201,9.0,4.56,3.0,2.0,4.0
1,Co-op City,40.874294,-73.829939,6.0,4.83,2.0,2.0,2.0
2,Eastchester,40.887556,-73.827806,1.0,4.00,0.0,1.0,0.0
3,Fieldston,40.895437,-73.905643,17.0,5.00,8.0,2.0,7.0
4,Riverdale,40.890834,-73.912585,15.0,4.33,5.0,3.0,7.0


Now that we got the italian restaurants data for each neighborhood, let us do the K-means to cluster the various neighborhoods of Bronx. Let us group them as 3 clusters.

In [168]:
# set number of clusters
kclusters = 3

Bronx_grouped_clustering = Bronx_data.drop(['Neighborhood','Latitude','Longitude'],1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:52] 

array([0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 1, 1, 2, 1, 1, 1, 2, 0, 2, 2,
       0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2,
       0, 0, 0, 2, 1, 1, 2, 1], dtype=int32)

In [169]:
Bronx_clustered = Bronx_data

# add clustering labels
Bronx_clustered['Cluster Labels'] = kmeans.labels_

Bronx_clustered.head() 

,Neighborhood,Latitude,Longitude,Count,Average Rating,High Rated Total,Mid Rated Total,Low Rated Total,Cluster Labels
0,Wakefield,40.894705,-73.847201,9.0,4.56,3.0,2.0,4.0,0
1,Co-op City,40.874294,-73.829939,6.0,4.83,2.0,2.0,2.0,0
2,Eastchester,40.887556,-73.827806,1.0,4.00,0.0,1.0,0.0,0
3,Fieldston,40.895437,-73.905643,17.0,5.00,8.0,2.0,7.0,2
4,Riverdale,40.890834,-73.912585,15.0,4.33,5.0,3.0,7.0,2


 Get the geographical coordinate of New York City

In [170]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


Let us view the map based on clusters of all Neighborhoods of Bronx.

In [171]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bronx_clustered['Latitude'], Bronx_clustered['Longitude'], Bronx_clustered['Neighborhood'], Bronx_clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters    

### Cluster 1

In [172]:
Bronx_clustered[Bronx_clustered['Cluster Labels']==0]

,Neighborhood,Latitude,Longitude,Count,Average Rating,High Rated Total,Mid Rated Total,Low Rated Total,Cluster Labels
0,Wakefield,40.894705,-73.847201,9.0,4.56,3.0,2.0,4.0,0
1,Co-op City,40.874294,-73.829939,6.0,4.83,2.0,2.0,2.0,0
2,Eastchester,40.887556,-73.827806,1.0,4.00,0.0,1.0,0.0,0
7,Norwood,40.877224,-73.879391,11.0,5.82,5.0,4.0,2.0,0
8,Williamsbridge,40.881039,-73.857446,4.0,5.00,1.0,2.0,1.0,0
11,City Island,40.847247,-73.786488,4.0,3.75,0.0,3.0,1.0,0
19,Melrose,40.819754,-73.909422,11.0,4.36,4.0,2.0,5.0,0
22,Longwood,40.815099,-73.895788,5.0,5.60,2.0,2.0,1.0,0
23,Hunts Point,40.809730,-73.883315,5.0,4.60,1.0,3.0,1.0,0
24,Morrisania,40.823592,-73.901506,10.0,5.20,3.0,3.0,4.0,0


### Cluster 2

In [173]:
Bronx_clustered[Bronx_clustered['Cluster Labels']==1]

,Neighborhood,Latitude,Longitude,Count,Average Rating,High Rated Total,Mid Rated Total,Low Rated Total,Cluster Labels
12,Bedford Park,40.870185,-73.885512,42.0,4.45,12.0,13.0,17.0,1
13,University Heights,40.855727,-73.910416,49.0,4.47,11.0,19.0,19.0,1
15,Fordham,40.860997,-73.896427,42.0,4.69,14.0,12.0,16.0,1
16,East Tremont,40.842696,-73.887356,42.0,3.14,8.0,8.0,26.0,1
17,West Farms,40.839475,-73.877745,43.0,4.02,12.0,11.0,20.0,1
29,Parkchester,40.837938,-73.856003,32.0,4.66,11.0,9.0,12.0,1
30,Westchester Square,40.840619,-73.842194,37.0,3.84,8.0,13.0,16.0,1
31,Van Nest,40.843608,-73.866299,47.0,4.32,13.0,11.0,23.0,1
32,Morris Park,40.847549,-73.850402,34.0,3.56,5.0,12.0,17.0,1
33,Belmont,40.857277,-73.888452,40.0,4.18,9.0,14.0,17.0,1


### Cluster 3

In [174]:
Bronx_clustered[Bronx_clustered['Cluster Labels']==2]

,Neighborhood,Latitude,Longitude,Count,Average Rating,High Rated Total,Mid Rated Total,Low Rated Total,Cluster Labels
3,Fieldston,40.895437,-73.905643,17.0,5.00,8.0,2.0,7.0,2
4,Riverdale,40.890834,-73.912585,15.0,4.33,5.0,3.0,7.0,2
5,Kingsbridge,40.881687,-73.902818,19.0,4.63,7.0,4.0,8.0,2
6,Woodlawn,40.898273,-73.867315,14.0,4.93,7.0,2.0,5.0,2
9,Baychester,40.866858,-73.835798,14.0,4.93,5.0,4.0,5.0,2
10,Pelham Parkway,40.857413,-73.854756,30.0,4.90,12.0,9.0,9.0,2
14,Morris Heights,40.847898,-73.919672,18.0,3.89,6.0,3.0,9.0,2
18,High Bridge,40.836623,-73.926102,18.0,3.50,3.0,5.0,10.0,2
20,Mott Haven,40.806239,-73.916100,15.0,4.20,5.0,2.0,8.0,2
21,Port Morris,40.801664,-73.913221,14.0,4.79,2.0,9.0,3.0,2


After observing the Neighborhood clusters, 

The cluster 1 has decent number of italian restaurants with more number of low rated italian restaurants. 

The cluster 2 has lower number of italian restaurants but has decent average ratings.

The cluster 3 has high number of italian restaurants and is having decent average ratings.

The cluster 3 will be recommended since our client is looking for a job and it is easy for him to find one if there are plenty of options. Here, we can't only go by the total number of italian restaurants. Because, though the neighborhood has more number of restaurants, the ratings can be poor. So, I am going to find the best neighborhood in cluster 3 by means of following formula in such a way that count holds 20% weight and average rating holds 80% weight.

Restaurant_score = (((High_rated * 2) + Mid_rated - (Low_rated * 2) ) * 0.2 ) + (Average_rating * 0.8)

The best neighborhood is the one which has highest restaurant_score.

In [175]:
Bronx_cluster_3=Bronx_clustered[Bronx_clustered['Cluster Labels']==2].reset_index(drop=True)
Bronx_cluster_3=Bronx_cluster_3.drop('Cluster Labels',1)

Bronx_cluster_3['Score']=(0.2*((Bronx_cluster_3['High Rated Total']*2)
                                +Bronx_cluster_3['Mid Rated Total']
                                -(Bronx_cluster_3['Low Rated Total']*2)))+(0.8*Bronx_cluster_3['Average Rating'])

Bronx_cluster_3.head(Bronx_cluster_3.shape[0])

,Neighborhood,Latitude,Longitude,Count,Average Rating,High Rated Total,Mid Rated Total,Low Rated Total,Score
0,Fieldston,40.895437,-73.905643,17.0,5.00,8.0,2.0,7.0,4.800
1,Riverdale,40.890834,-73.912585,15.0,4.33,5.0,3.0,7.0,3.264
2,Kingsbridge,40.881687,-73.902818,19.0,4.63,7.0,4.0,8.0,4.104
3,Woodlawn,40.898273,-73.867315,14.0,4.93,7.0,2.0,5.0,5.144
4,Baychester,40.866858,-73.835798,14.0,4.93,5.0,4.0,5.0,4.744
5,Pelham Parkway,40.857413,-73.854756,30.0,4.90,12.0,9.0,9.0,6.920
6,Morris Heights,40.847898,-73.919672,18.0,3.89,6.0,3.0,9.0,2.512
7,High Bridge,40.836623,-73.926102,18.0,3.50,3.0,5.0,10.0,1.000
8,Mott Haven,40.806239,-73.916100,15.0,4.20,5.0,2.0,8.0,2.560
9,Port Morris,40.801664,-73.913221,14.0,4.79,2.0,9.0,3.0,5.232


Based on the scores of the restaurants, one neighborhood which stands out is West Farms which has 11.08.

Let us get the top 5 hotels available in West Farms neighborhood for our client to stay. The foursquare category ID is 4bf58dd8d48988d1fa931735 for hotels. We will use it in the url. Below function returns the list of hotels.

In [176]:
def get_hotels(neighborhood_latitude,neighborhood_longitude):
    LIMIT = 100 # limit of number of venues returned by Foursquare API

    radius = 2000 # define radius
    # create URL
    url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d1fa931735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT) 
    HotelsList=[]
    results = requests.get(url).json()
    Hotelscount= len(results['response']['venues'])
    
    for i in range(Hotelscount):
        Hotel=results['response']['venues'][i]['name']
        HotelsList.append(Hotel)
    
    return HotelsList

In [177]:
# Let us set the latitude and longitude of West Farms and get the list of hotels.
neighborhood_latitude=40.839475
neighborhood_longitude=-73.877745
HotelsList=get_hotels(neighborhood_latitude,neighborhood_longitude)
print(HotelsList)

['Howard Johnson Hotel', 'Sheridan Hotel', 'Bronx Park Hotel', 'Elder', 'Coblentz', 'Best Western Bronx Inn', 'Richard Merino', 'Coco Beach', 'Hotel Paradise', 'Hotel Royal', 'Jet Set Hotel', 'Super 8']


Below function returns rating of hotels.

In [178]:
def get_rating(venueid,neighborhood_latitude,neighborhood_longitude):
    venue_url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
    venueid,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude
    )
    
    venue_results = requests.get(venue_url).json()
    
    try:
        hotel_rating = venue_results['response']['venue']['rating']
    except:
        hotel_rating = 0
    
    hotel_name = venue_results['response']['venue']['name']
    
    return hotel_name, hotel_rating

Since the venue details such as rating end_point is premium call in Foursquare and my quota has exceeded, instead of using the above function, I will be generating random number for each hotels as rating.

In [179]:
HotelRatingList=[]
for hotel in HotelsList:
    hotel_temp=['',0]
    HotelRating=random.randint(0,10)
    print('The hotel {} has {} rating'.format(hotel,HotelRating))
    Hotels_df['hotel']=hotel
    Hotels_df['Rating']=HotelRating
    hotel_temp[0]=hotel
    hotel_temp[1]=HotelRating
    HotelRatingList.append(hotel_temp)

HotelRatingList


The hotel Howard Johnson Hotel has 3 rating
The hotel Sheridan Hotel has 0 rating
The hotel Bronx Park Hotel has 5 rating
The hotel Elder has 2 rating
The hotel Coblentz has 0 rating
The hotel Best Western Bronx Inn has 10 rating
The hotel Richard Merino has 7 rating
The hotel Coco Beach has 4 rating
The hotel Hotel Paradise has 10 rating
The hotel Hotel Royal has 4 rating
The hotel Jet Set Hotel has 6 rating
The hotel Super 8 has 6 rating


[['Howard Johnson Hotel', 3],
 ['Sheridan Hotel', 0],
 ['Bronx Park Hotel', 5],
 ['Elder', 2],
 ['Coblentz', 0],
 ['Best Western Bronx Inn', 10],
 ['Richard Merino', 7],
 ['Coco Beach', 4],
 ['Hotel Paradise', 10],
 ['Hotel Royal', 4],
 ['Jet Set Hotel', 6],
 ['Super 8', 6]]

Let us create a dataframe which consists of hotels and ratings

In [180]:
hotels_df = pd.DataFrame(HotelRatingList,columns=['Hotel','Rating'])
hotels_df.head(15)

,Hotel,Rating
0,Howard Johnson Hotel,3
1,Sheridan Hotel,0
2,Bronx Park Hotel,5
3,Elder,2
4,Coblentz,0
5,Best Western Bronx Inn,10
6,Richard Merino,7
7,Coco Beach,4
8,Hotel Paradise,10
9,Hotel Royal,4


Let us get the top 5 hotels to stay based on rating in West Farm, Bronx, New York City.

In [181]:
hotels_df=hotels_df.sort_values(by='Rating',ascending=False).reset_index(drop=True)
hotels_df=hotels_df.head()
hotels_df

,Hotel,Rating
0,Best Western Bronx Inn,10
1,Hotel Paradise,10
2,Richard Merino,7
3,Jet Set Hotel,6
4,Super 8,6


# Results

After studying the Bronx's neighborhoods for italian restaurants for our client Michael to get a job, we found that the West Farms is the best neighborhoods having more number of italian restaurants and good ratings. Also, in West Farms neighborhood, Elder, Coblentz, Super 8, Best Western Bronx Inn and Hotel Paradise are the top 5 hotels to stay.